IHH ~ M&A

https://cran.r-project.org/web/packages/plm/plm.pdf

https://cran.r-project.org/web/packages/plm/vignettes/plmFunction.html

https://cran.r-project.org/web/packages/plm/vignettes/plmModelComponents.html

https://datascienceplus.com/working-with-panel-data-in-r-fixed-vs-random-effects-plm/

Тема: 
- влияние сделок слияний и поглощений на рыночную концентрацию
- нужно выделить гипотезы: построить модель (мне сказали re, fe, pool), понять есть ли динамика, сделать выводы


Гипотезы:
1. сделки слияний и поглощений положительно влияют на индекс концентрации
2. сделки слияний и поглощений способствют тому что компании-лидеры увеличивают свои доли на рынках
3. для отраслей с высокой концентрацией сделки оказывают меньшее воздействие на повышение концентрации, чем для тех, где уровень концентрации ниже

- TR - выручка
- CR - концентрация
- IHH - индекс херфиндаля хиршмана
- M&A - mergers and acquisitions


- проверить, влияет ли лаг, т.е. сделки текущего года влияют на IHH следующего
- ?дамми на тип сделки (внутренняя/внешняя)

In [270]:
import pandas as pd
import numpy as np

In [271]:
raw = pd.read_csv('raw_data.csv', index_col=0, header=0)
raw

,year,otrasl,TR,TR10,TR50,TR100,TR10%,CR(10),CR(50),CR(100),CR10%,IHH,MA,real_prices,real_gdp_change,market_type,type_1,type_2,type_3,type_4
0,2013,ДОБЫЧА,1.212809e+13,6897734823000,10021979664000,10917975429000,11332003071000,0.568740,0.826344,0.900222,0.934360,985.881015,1.670287e+04,1.670287e+04,1289.149400,3,0,0,1,0
1,2014,ДОБЫЧА,1.363228e+13,7870674895000,11274710723000,12296084533000,12742057761000,0.577356,0.827060,0.901983,0.934697,1112.472804,8.834030e+03,8.217709e+03,408.890698,4,0,0,0,1
2,2015,ДОБЫЧА,1.436711e+13,7706976156000,11645533807000,12794578175000,13311848403000,0.536432,0.810569,0.890546,0.926550,842.981755,6.636370e+03,5.716084e+03,-1971.472868,3,0,0,1,0
3,2016,ДОБЫЧА,1.462882e+13,7647183873000,11684568749000,12929957474000,13479545832000,0.522748,0.798736,0.883869,0.921437,842.693180,3.311652e+04,2.774721e+04,497.072160,3,0,0,1,0
4,2017,ДОБЫЧА,1.736623e+13,9182911407000,13845695452000,15331338131000,16019846244000,0.528780,0.797277,0.882825,0.922471,906.929965,8.821010e+03,7.018832e+03,1216.097447,3,0,0,1,0
5,2013,IT,6.690939e+11,150843772000,278656460034,350202574034,499589016034,0.225445,0.416468,0.523398,0.746665,86.418892,3.435151e+04,3.435151e+04,1289.149400,0,0,0,0,0
6,2014,IT,7.520101e+11,165284287000,314178600034,392496577034,560714736750,0.219790,0.417785,0.521930,0.745621,91.337393,5.189821e+04,4.827741e+04,408.890698,0,0,0,0,0
7,2015,IT,8.515278e+11,197416903000,368862369000,458440253000,645250453000,0.231838,0.433177,0.538374,0.757756,98.720693,1.097405e+05,9.452243e+04,-1971.472868,0,0,0,0,0
8,2016,IT,9.888551e+11,245445165000,461105323000,557451972000,765391183507,0.248211,0.466302,0.563735,0.774017,110.627066,3.074769e+05,2.576245e+05,497.072160,1,1,0,0,0
9,2017,IT,2.755628e+12,52273333000,156699011034,225147116034,383205146750,0.069511,0.208374,0.299394,0.509574,13.924398,4.619760e+04,3.675918e+04,1216.097447,0,0,0,0,0


In [272]:
raw[1::5]

,year,otrasl,TR,TR10,TR50,TR100,TR10%,CR(10),CR(50),CR(100),CR10%,IHH,MA,real_prices,real_gdp_change,market_type,type_1,type_2,type_3,type_4
1,2014,ДОБЫЧА,1.363228e+13,7870674895000,11274710723000,12296084533000,12742057761000,0.577356,0.827060,0.901983,0.934697,1112.472804,8834.03000,8217.708986,408.890698,4,0,0,0,1
6,2014,IT,7.520101e+11,165284287000,314178600034,392496577034,560714736750,0.219790,0.417785,0.521930,0.745621,91.337393,51898.21000,48277.409395,408.890698,0,0,0,0,0
11,2014,ПИЩЕВАЯ,4.157523e+12,733637927000,1586095583000,2001297529000,3211155171000,0.176460,0.381500,0.481368,0.772372,49.753877,17067.50000,15876.746493,408.890698,0,0,0,0,0
16,2014,СМИ,2.940133e+11,140870401000,223710525000,248340528000,249536249000,0.479129,0.760886,0.844658,0.848724,364.525355,28891.66020,26875.962977,408.890698,2,0,1,0,0
21,2014,МАШИНОСТРОЕНИЕ,4.193064e+12,1168889592000,2239981084000,2704132137000,3610982831400,0.278767,0.534211,0.644906,0.861180,117.119562,48730.74486,45330.925451,408.890698,1,1,0,0,0
26,2014,ЛЕГКАЯ,3.210211e+11,97712719000,161602265000,196973643000,227799007000,0.304381,0.503401,0.613585,0.709607,198.029611,265.34760,246.834977,408.890698,1,1,0,0,0
31,2014,ЛЕСНАЯ,9.989583e+10,23413410000,49911535000,64918985000,58752562000,0.234378,0.499636,0.649867,0.588138,103.380199,30328.37637,28212.443135,408.890698,1,1,0,0,0
36,2014,МЕТАЛЛУРГИЯ,3.395286e+12,1779300127000,2936478831000,3214533002000,2975922488000,0.524050,0.864869,0.946764,0.876487,382.002688,33244.11574,30924.758828,408.890698,2,0,1,0,0
41,2014,СЕЛЬХОЗ,1.560797e+12,130277834000,329573301000,472361816000,990155286000,0.083469,0.211157,0.302641,0.634391,17.360622,55169.14079,51320.130967,408.890698,0,0,0,0,0
46,2014,СПОРТ,6.237502e+10,28316880000,43030745000,48330536000,43030745000,0.453978,0.689871,0.774838,0.689871,288.005237,3753.93656,3261.116093,408.890698,2,0,1,0,0


- проранжировать отрасли по степени концентрации и ввести например переменную "ранг", или как-то ещё проконтролировать эту нелинейность

In [ ]:
a = '4'
raw.loc[raw[raw.IHH>=1000].index, 'market_type'] =   a 

In [210]:
a = '3'
raw.loc[raw[(raw.IHH<1000) &  (raw.IHH>=400)].index, 'market_type'] =   a 

In [212]:
a = '2'
raw.loc[raw[(raw.IHH<400) &  (raw.IHH>=200)].index, 'market_type'] =   a 

In [213]:
a = '1'
raw.loc[raw[(raw.IHH<200) &  (raw.IHH>=100)].index, 'market_type'] =   a 

In [214]:
a = '0'
raw.loc[raw[(raw.IHH<100)].index, 'market_type'] =   a 

In [218]:
df2 = pd.get_dummies(raw.market_type,drop_first=True)

In [221]:
raw = pd.concat([raw , df2], axis=1) 

- привести цены сделок к одному году

In [ ]:
pr = pd.read_csv('raw_prices.csv', index_col=None, header=0)
pr.iloc[1:,1:] = pr.iloc[1:,1:].div(pr.iloc[0,1:], axis = 'columns')
pr = pr.drop(columns=['2012', '2018'])
pr = pr.drop(index=0)

- добавить в регрессоры темп роста ввп (потому что в период роста могут чаще покупать)

In [115]:
pr2 = pd.read_csv('raw_def.csv', index_col=0, header=0)

In [166]:
pr2 = pr2.drop(index=2018)

In [226]:
raw.to_csv('raw_data.csv')

In [275]:
dat2 = pd.read_excel('data_ma.xlsx', header=1)

In [276]:
np.unique(np.array(dat2['Формат сделки'].values))

array(['Внутренняя', 'Внутренняя ', 'Трансграничная (in-out)',
       'Трансграничная (out-in)'], dtype=object)

In [277]:
dat2['Формат сделки'] = dat2['Формат сделки'].apply(lambda x: x[:5])

In [283]:
np.unique(dat2['Отрасль'].values)

array(['IT', 'Добыча полезных ископаемых', 'Легкая промышленность',
       'Лесная', 'Машиностроение', 'Металлургия',
       'Пищевая промышленность', 'Прочие производства', 'СМИ', 'Связь',
       'Сельское хозяйство', 'Спорт', 'Страхование',
       'Строительство и девелопмент', 'ТЭК', 'Торговля', 'Торговля ',
       'Транспорт', 'Услуги', 'Финансовые институты',
       'Химическая и нефтехимическая промышленность', 'Электроэнергетика'],
      dtype=object)

In [288]:
dat2['Отрасль'][dat2['Отрасль']=='Торговля '] #= 'Торговля'

Series([], Name: Отрасль, dtype: object)

In [287]:
dat2.loc[dat2['Отрасль']=='Торговля ', 'Отрасль']

Series([], Name: Отрасль, dtype: object)

In [289]:
dat2.index = pd.MultiIndex.from_arrays([dat2['Отрасль'].values, dat2['Формат сделки'].values])

In [292]:
dat2.groupby(level=[0,1]).count().iloc[:,0].to_csv('123.csv')

In [293]:
dat2.groupby(level=[0,1]).count().iloc[:,0].groupby(level=0).sum().to_csv('124.csv')

In [294]:
dat2.groupby(level=[0,1]).sum().iloc[:,0].to_csv('125.csv')

In [316]:
raw = pd.read_csv('raw_data.csv', index_col=0, header=0)

In [317]:
raw['merge_index'] = raw['year'].apply(lambda x: str(x)) + raw['otrasl']

In [318]:
rawty = pd.read_csv('vse_typy_sdelok.csv', index_col=0, header=0)

In [320]:
rawty['merge_index'] = rawty['year'].apply(lambda x: str(x)) + rawty['otrasl']

In [321]:
np.unique(rawty['otrasl'])

array(['IT', 'ДОБЫЧА', 'ЛЕГКАЯ', 'ЛЕСНАЯ', 'МАШИНОСТРОЕНИЕ',
       'МЕТАЛЛУРГИЯ', 'НЕФТЕХИМ', 'ПИЩЕВАЯ', 'Прочие производства',
       'СВЯЗЬ', 'СЕЛЬХОЗ', 'СМИ', 'СПОРТ', 'СТРОИТЕЛЬСТВО', 'Страхование',
       'ТОРГОВЛЯ', 'ТЭК', 'ТранСПОРТ', 'УСЛУГИ', 'ФИНАНСОВЫЕ ИНСТИТУТЫ',
       'ЭЛЕКТРОЭНЕРГЕТИКА'], dtype=object)

In [322]:
np.unique(raw['otrasl'])

array(['IT', 'ДОБЫЧА', 'ЛЕГКАЯ', 'ЛЕСНАЯ', 'МАШИНОСТРОЕНИЕ',
       'МЕТАЛЛУРГИЯ', 'НЕФТЕХИМ', 'ПИЩЕВАЯ', 'СВЯЗЬ', 'СЕЛЬХОЗ', 'СМИ',
       'СПОРТ', 'СТРОИТЕЛЬСТВО', 'ТОРГОВЛЯ', 'ТЭК', 'УСЛУГИ',
       'ФИНАНСОВЫЕ ИНСТИТУТЫ', 'ЭЛЕКТРОЭНЕРГЕТИКА'], dtype=object)

In [292]:
dat2.groupby(level=[0,1]).count().iloc[:,0].to_csv('123.csv')

In [347]:
rawty.index = pd.MultiIndex.from_arrays([rawty['year'].values, rawty['otrasl'].values])

In [355]:
# rawty.to_csv('rawty_2.csv')

In [376]:
raw1 = pd.read_csv('rawty_1.csv', index_col=None, header=0)
raw2 = pd.read_csv('rawty_2.csv', index_col=None, header=0)
raw3 = pd.read_csv('rawty_3.csv', index_col=None, header=0)

In [382]:
rawfull = raw.set_index('merge_index').join(raw1.iloc[:,[-1,3]].set_index('merge_index'), rsuffix='_inout' )
rawfull = rawfull.join(raw2.iloc[:,[-1,3]].set_index('merge_index'), rsuffix='_in' )
rawfull = rawfull.join(raw3.iloc[:,[-1,3]].set_index('merge_index'), rsuffix='_outin' )


In [384]:
rawfull.to_csv('raw_full.csv')